In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
from FileFinder import FileFinder as ff

path = ff.get_path('constituents.csv')

#path = "Users/rhys/Desktop/740/Reference_Data/constituents.csv"

path


'/Users/rhys/Desktop/740/Reference_Data/constituents.csv'

In [3]:

sp500_Data = pd.read_csv(path)

# .sample(100)
symbols = sp500_Data['Symbol'].to_list()
sector = sp500_Data['Sector'].to_list()


def sort_tick_frame(calls_df, puts_df, exp, tick):
    
    now = datetime.datetime.now()
    
    df = pd.concat([calls_df, puts_df], axis = 0)
    df['mark'] = (df['bid'] + df['ask']) / 2 
    df['expirationDate'] = exp
    df['days_to_expiration'] = [(pd.to_datetime(i) - now).days for i in df['expirationDate']]
    df['ticker'] = tick
    df['sector'] = sector[num]
    #df['CALL'] = df['contractSymbol'].str[4:].apply(lambda x: "C" in x)
    df = df.drop(columns = ['contractSize', 'currency', 'lastTradeDate'])
    
    return df

In [ ]:


options = pd.DataFrame()

length = len(symbols)
count = 1
for num, tick in enumerate(symbols):
    
    
    print(f"{count} of {length}")
    count += 1
    
    if '.' in tick:
        tick.replace('.', '-')
    
    ticker = yf.Ticker(tick)
    exprys = ticker.options
    
    for exp in exprys:
        
        ticker_options = ticker.option_chain(exp)
        
        calls = ticker_options.calls
        calls['inTheMoney'] = calls['inTheMoney'].astype("int32")
        calls['CALL'] = 1
        puts = ticker_options.puts
        puts['inTheMoney'] = puts['inTheMoney'].astype("int32")
        puts['CALL'] = 0
        
        this_ticker = sort_tick_frame(ticker_options.calls, ticker_options.puts, exp, tick)
        options = pd.concat([options, this_ticker], axis = 0, ignore_index=True)

In [ ]:

options_per_exp_date = {}
call_options_per_exp_date = {}

time_buckets = ['<1M', '1-3M', '3-6M', '6M-1Y', '1-2Y', '2+Y']

for exp in options.days_to_expiration.sort_values().unique():
    
    if exp <= 30:
        
        key = time_buckets[0]
        
    elif exp <= 90:
        key = time_buckets[1]
    
    elif exp <= 180:
        key = time_buckets[2]
    
    elif exp <= 365:
        key = time_buckets[3]
    
    elif exp <= 730:
        key = time_buckets[4]
    
    elif exp > 730:
        key = time_buckets[5]
        
    options_per_exp_date[key] = {}
    options_per_exp_date[key]['put'] = options.loc[ (options.days_to_expiration == exp) & (options.CALL == 0) & (options.inTheMoney == 1) ]['mark'].mean()
    options_per_exp_date[key]['call'] = options.loc[ (options.days_to_expiration == exp) & (options.CALL == 1) & (options.inTheMoney == 1) ]['mark'].mean()


In [ ]:


pd.DataFrame(options_per_exp_date).T.plot.bar()

